In [ ]:
def plot_ROI_contour(user_stock_df, ROI_df, strategy_df, relative=False):

    df = ROI_df.copy()
    stock = str(user_stock_df.stock)
    strategy = str(user_stock_df.strategy)
    x_column_name = str(df.columns[0])
    y_column_name = str(df.columns[1])
    z_column_name = str(df.columns[2])
    x_value = int(strategy_df.x_value)
    y_value = int(strategy_df.y_value)
    title=z_column_name + ' of ' + stock + ' by ' + strategy + '(' + x_column_name + ', ' + y_column_name + ')'
    if relative>0:
        title=title+' relative to '+ strategy + '(' + str(x_value) + ', ' + str(y_value) + ')'
        common_return = float(df[(df[x_column_name]==x_value) & (df[y_column_name]==y_value)][z_column_name])
        df[z_column_name] = df[z_column_name]-common_return
        
    colorscale = [[0, 'red'], [0.5, 'white'], [1, 'green']]
    df_heatmap = df.pivot(index=y_column_name, columns=x_column_name, values=z_column_name)
    fig = go.Figure(go.Contour(
            z=df_heatmap.to_numpy(),
            x0=int(strategy_df.x0),
            dx=int(strategy_df.dx),
            y0=int(strategy_df.y0),
            dy=int(strategy_df.dy),
            colorscale=colorscale,
            name='',
            colorbar=dict(title=z_column_name),
            hovertemplate=x_column_name + ': %{x}<br>' + y_column_name + ': %{y}<br>' + z_column_name + ': %{z}',
            contours=dict(
                start=-1,
                end=1,
                size=0.1,
                coloring ='heatmap',
                showlabels = True, # show labels on contours
                labelfont = dict( # label font properties
                    size = 12,
                    color = 'black',
                )
        ),
        ))

    fig.update_layout(
    title=title,
    autosize=False,
    width=750,
    height=750
)
    fig.update_xaxes(title_text=x_column_name)
    fig.update_yaxes(title_text=y_column_name)
    fig.show()
    # save the image
    import os
    folder_name = "images"
    image_name = strategy + '_' + stock + '_' + period
    if relative:
        image_name = image_name + '_' + 'normalized'
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    fig.write_image(folder_name+'/' + image_name + '.png')

In [ ]:
def Tuning(user_email):
    optimize_ROI_df = pd.DataFrame(columns = ['strategy', 'stock', 'x', 'y', 'z'])
    best_ROI_df = pd.DataFrame(columns = ['strategy', 'stock', 'x', 'y', 'z'])
    z_name = "Return"
    for _, user_stock_row in user_stock_df[user_stock_df.email == user_email].iterrows():
        strategy_df=get_strategy_df(str(user_stock_row.strategy)).copy()
        for _, strategy_row in strategy_df.iterrows():
            x_name=str(strategy_row.x_name)
            y_name=str(strategy_row.y_name)
            x_value=int(strategy_row.x_value)
            y_value=int(strategy_row.y_value)
            x0=int(strategy_row.x0)
            x_max=int(strategy_row.x_max)
            dx=int(strategy_row.dx)
            y0=int(strategy_row.y0)
            y_max=int(strategy_row.y_max)
            dy=int(strategy_row.dy)
        column_names = [x_name, y_name, z_name]
        ROI_df = pd.DataFrame(columns = column_names)
        max_x = 0
        max_y = 0
        max_ROI = -100
        print(user_stock_row.stock)
        for x in range(x0, x_max+1, dx):
            for y in range(y0, y_max+1, dy):
                position = backtesting(user_stock_row, str(x), str(y))
                ROI = ((position.value[-1]-position.value[0])/position.value[0]).round(2)
                new_row = {x_name:x, y_name:y, z_name:ROI}
                ROI_df = ROI_df.append(new_row, ignore_index=True)
                if ROI>max_ROI:
                    max_ROI = ROI
                    max_x = x
                    max_y = y
                new_optimize_row = {'strategy':str(user_stock_row.strategy), 'stock':str(user_stock_row.stock), 'x':x, 'y':y, 'z':ROI}
                optimize_ROI_df = optimize_ROI_df.append(new_optimize_row, ignore_index=True)
        plot_ROI_contour(user_stock_row, ROI_df, strategy_df)
        new_best_row = {'strategy':str(user_stock_row.strategy), 'stock':str(user_stock_row.stock), 'x':max_x, 'y':max_y, 'z':max_ROI}
        best_ROI_df = best_ROI_df.append(new_best_row, ignore_index=True)
        print(str(user_stock_row.strategy) + '(' + str(max_x) + ', ' + str(max_y) + ') is better for ' + str(user_stock_row.stock))
        plot_ROI_contour(user_stock_row, ROI_df, strategy_df, relative=True)
    return best_ROI_df, optimize_ROI_df

In [ ]:
best_ROI_df, optimize_ROI_df = Tuning(user_email)

In [ ]:
def get_ROI_from_default(x=0, y=0):
    if x==0 or y==0:
        strategy = optimize_ROI_df.strategy[0]
        strategy_row = strategy_df[strategy_df['strategy']==strategy].head(1)
        x = int(strategy_row['x_value'])
        y = int(strategy_row['y_value'])
    df = optimize_ROI_df[(optimize_ROI_df.x==x) & (optimize_ROI_df.y==y)].copy()
    return(df)